In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib qt 

import sys
import napari
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

from utils import  load_video_as_array
from consts import HERE

sys.path.append(str(HERE.parent))


In [7]:
pupil_umbrella = Path("/Volumes/MarcBusche/James/Regular2p")
mouse = "J029"
date = "2024-10-25"
pupil_folder = pupil_umbrella / date / mouse
mp4_paths = sorted(list(pupil_folder.glob("*.mp4")))

In [8]:
show_napari = True
print(mouse)

if show_napari:
    # Inspect the binary form of the video
    video = load_video_as_array(mp4_paths[0], 500)

    viewer = napari.Viewer()
    binarised = (video > 100).astype('int')

    viewer.add_image(video, name="Video")
    viewer.add_image(binarised, name="Binanrised")

J029


In [32]:
shape = [layer for layer in viewer.layers if layer.name == "Shapes"][0]

[key for key in shape.__dict__.keys() if key[0] != "_"]

top_left =  [shape.corner_pixels[0, 1], shape.corner_pixels[0, 2]]
bottom_right = [shape.corner_pixels[1,1], shape.corner_pixels[1,2]]


bottom_right[0] = bottom_right[0] +  400 - (bottom_right[0] - top_left[0])
bottom_right[1] = bottom_right[1] +  400 - (bottom_right[1] - top_left[1])

assert (bottom_right[0] - top_left[0]) == 400
assert (bottom_right[1] - top_left[1]) == 400


save_path=HERE.parent / "cache" / f"{mouse}_{date}_corner_pixels.npy"
np.save(save_path,(top_left,bottom_right))


In [33]:
print(mouse)
save_path=HERE.parent / "cache" / f"{mouse}_{date}_corner_pixels.npy"
video = load_video_as_array(mp4_paths[0], 700)
top_left,bottom_right = np.load(save_path)

viewer = napari.Viewer()

cropped_video = video[:, top_left[0]: bottom_right[0], top_left[1]: bottom_right[1]]
binarised = (cropped_video > 200).astype(int)

diffed = np.abs(np.diff(binarised, axis=0))
viewer.add_image(binarised, name="Cropped Binarised")
viewer.add_image(diffed, name="Cropped Diffed")
viewer.add_image(cropped_video, name="Cropped Video")

J029


<Image layer 'Cropped Video' at 0x2a0761a60>

In [ ]:
# plt.clf()
# plt.plot(np.sum(diffed, (1,2)))
# plt.show()

In [17]:
x = np.load(HERE.parent / "cache" / f"{mouse}_{date}_corner_pixels.npy")